In [1]:
import numpy as np
import torch
import json
import torch.nn.functional as F
import torch.nn as nn
import torch.utils as utils

In [38]:
def read_data(filename):
    """
    Read data from a file.
    """
    with open(filename, 'r') as f:
        data = f.readlines()
    return data


def convert_str_to_list(line):
    """
    Convert string to list.
    """
    return json.loads(line)

In [43]:
data = read_data('D:/A-bus/bus_pytorch/data/dataset/train.txt')
data = [convert_str_to_list(line) for line in data]
data = np.array(data)
data[0].shape
# l = np.array(l)
# max_len = 50
# route_len = int(len(l)/30)
# l = l.reshape(30, route_len)
# l = torch.Tensor(l)
# print(l.shape)
# l = F.pad(input=l, pad=(0, max_len-route_len, 0, 0), mode='constant', value=0)
# print(l.shape)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (345,) + inhomogeneous part.

In [19]:
causal_conv = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=(3), padding=0)
x = torch.randn(16, 1, 29, 50)
# for i in range (16):
#     x[i] = F.pad(input=x[i], pad=(0, 0, 0, 0), mode='constant', value=0)
#     x[i] = causal_conv(x[i])
#     print(x[i].shape)
causal_conv(x[0,:,:, 0]).shape

torch.Size([16, 27])

In [8]:
x = torch.randn(16, 1, 29, 50)
x[0,:,:, 0].shape

torch.Size([1, 29])

In [9]:
import torch
from torch import nn
import numpy as np
import math

def t2v(tau, f, out_features, w, b, w0, b0, arg=None):
    if arg:
        v1 = f(torch.matmul(tau, w) + b, arg)
    else:
        #print(w.shape, t1.shape, b.shape)
        v1 = f(torch.matmul(tau, w) + b)
        print('v1:', v1.shape)
    v2 = torch.matmul(tau, w0) + b0
    print('v2:',v2.shape)
    return torch.cat([v1, v2], 1)

class SineActivation(nn.Module):
    def __init__(self, in_features, out_features):
        super(SineActivation, self).__init__()
        self.out_features = out_features
        self.w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.parameter.Parameter(torch.randn(1))
        self.w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.b = nn.parameter.Parameter(torch.randn(out_features-1))
        self.f = torch.sin

    def forward(self, tau):
        return t2v(tau, self.f, self.out_features, self.w, self.b, self.w0, self.b0)

class CosineActivation(nn.Module):
    def __init__(self, in_features, out_features):
        super(CosineActivation, self).__init__()
        self.out_features = out_features
        self.w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.parameter.Parameter(torch.randn(1))
        self.w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.b = nn.parameter.Parameter(torch.randn(out_features-1))
        self.f = torch.cos

    def forward(self, tau):
        return t2v(tau, self.f, self.out_features, self.w, self.b, self.w0, self.b0)

In [10]:
sineact = SineActivation(1, 64)
cosact = CosineActivation(1, 64)

print(sineact(torch.Tensor([[7]])).shape)
print(cosact(torch.Tensor([[7]])).shape)

v1: torch.Size([1, 63])
v2: torch.Size([1, 1])
torch.Size([1, 64])
v1: torch.Size([1, 63])
v2: torch.Size([1, 1])
torch.Size([1, 64])


In [6]:
sineact(torch.Tensor([[7]]))

tensor([[-0.9098,  0.7401,  0.6955, -0.9861,  0.7383,  0.8965,  0.5066, -0.2459,
         -0.4764,  0.1394, -0.8101, -0.4006, -0.0696,  0.3543,  0.0761,  0.5986,
          0.1463, -0.2179, -0.6638,  0.6862,  0.8209,  0.9999,  0.2062, -0.3347,
         -0.9476, -0.8814,  0.1271, -0.9184,  0.7954, -0.8993, -1.0000, -0.9898,
          0.9873,  0.9461,  0.8175, -0.9950,  0.9294,  0.9425, -0.8157,  0.7968,
         -0.5311,  0.9614,  0.9470, -0.5990,  0.6272,  0.6897,  0.9307, -0.8034,
          0.5433, -0.9946,  0.7017,  0.5916, -0.4506, -0.8020,  0.5983,  0.0486,
          0.5446,  0.9905, -0.0303,  0.2498,  0.9485,  0.6191,  0.2212, 10.2144]],
       grad_fn=<CatBackward0>)

In [24]:
a = torch.randn((1))
b = torch.randn((1))
c = torch.mul(a, b)
print(c)

tensor([0.8603])


In [23]:
a = torch.ones((4,1))
b = torch.randn((1,1))
c = torch.mul(a, b)
print(a,b,c)
print(a.shape, b.shape, c.shape)

tensor([[1.],
        [1.],
        [1.],
        [1.]]) tensor([[-0.9848]]) tensor([[-0.9848],
        [-0.9848],
        [-0.9848],
        [-0.9848]])
torch.Size([4, 1]) torch.Size([1, 1]) torch.Size([4, 1])


In [49]:
# a = torch.ones((4,3))
# b = torch.randn((1,3))
# c = torch.mul(a, b)
# print(a,b,c)
# print(a.shape, b.shape, c.shape)

a = torch.ones((3,4,1))
b = torch.randn((3,1,7))
c = torch.mul(a, b)
print(a,b,c)
print(a.shape, b.shape, c.shape)

tensor([[[1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.]]]) tensor([[[ 1.8683, -0.8744,  1.9455,  1.0013, -0.6271,  0.1458, -0.1026]],

        [[ 2.3816, -1.9624, -2.3535,  0.0988,  3.1183,  0.1993,  1.1589]],

        [[ 0.2284, -0.3128, -0.8863,  0.9559,  1.2712, -0.4170,  0.0057]]]) tensor([[[ 1.8683, -0.8744,  1.9455,  1.0013, -0.6271,  0.1458, -0.1026],
         [ 1.8683, -0.8744,  1.9455,  1.0013, -0.6271,  0.1458, -0.1026],
         [ 1.8683, -0.8744,  1.9455,  1.0013, -0.6271,  0.1458, -0.1026],
         [ 1.8683, -0.8744,  1.9455,  1.0013, -0.6271,  0.1458, -0.1026]],

        [[ 2.3816, -1.9624, -2.3535,  0.0988,  3.1183,  0.1993,  1.1589],
         [ 2.3816, -1.9624, -2.3535,  0.0988,  3.1183,  0.1993,  1.1589],
         [ 2.3816, -1.9624, -2.3535,  0.0988,  3.1183,  0.1993,  1.1589],
         [ 2.3816, -1.9624, -2.3535,  0.0988,  3.1183,  0.1993,  1

In [57]:
a = torch.ones((2,4,3))# bz, days, stations
b = torch.randn((2,1,3))
c1 = torch.mul(a, b) # 2,4,3
print(a.shape, b.shape, c1.shape)
print(c1)

a = torch.ones((2,4,3)) # bz, days, stations
a = a.permute(0,2,1) # bz, stations, days
a = a.unsqueeze(-1) # bz, stations, days, 1
b = torch.randn((2,3,1,7))
c2 = torch.mul(a, b)  # 2,3,4,7
print(a.shape, b.shape, c2.shape)
print(c2)

c1_ = c1.permute(0,2,1).unsqueeze(-1)
print(c1_.shape,c2.shape)
c = torch.cat([c1_, c2], dim=3)
print('--------')
print(c)
print(c.shape)

torch.Size([2, 4, 3]) torch.Size([2, 1, 3]) torch.Size([2, 4, 3])
tensor([[[ 0.4222, -1.5881,  0.8632],
         [ 0.4222, -1.5881,  0.8632],
         [ 0.4222, -1.5881,  0.8632],
         [ 0.4222, -1.5881,  0.8632]],

        [[-2.5318,  0.5178,  0.1337],
         [-2.5318,  0.5178,  0.1337],
         [-2.5318,  0.5178,  0.1337],
         [-2.5318,  0.5178,  0.1337]]])
torch.Size([2, 3, 4, 1]) torch.Size([2, 3, 1, 7]) torch.Size([2, 3, 4, 7])
tensor([[[[ 0.6064, -0.4609, -2.3317,  1.4858, -2.0842, -0.2221,  1.2128],
          [ 0.6064, -0.4609, -2.3317,  1.4858, -2.0842, -0.2221,  1.2128],
          [ 0.6064, -0.4609, -2.3317,  1.4858, -2.0842, -0.2221,  1.2128],
          [ 0.6064, -0.4609, -2.3317,  1.4858, -2.0842, -0.2221,  1.2128]],

         [[ 0.2640, -1.1525,  0.2152, -2.1817, -0.9160, -0.4981,  1.3182],
          [ 0.2640, -1.1525,  0.2152, -2.1817, -0.9160, -0.4981,  1.3182],
          [ 0.2640, -1.1525,  0.2152, -2.1817, -0.9160, -0.4981,  1.3182],
          [ 0.2640, -1.1

In [69]:
transformer_model = nn.Transformer(nhead=16, num_encoder_layers=12)
src = torch.rand((10, 32, 512))
tgt = torch.rand((20, 32, 512))
out = transformer_model(src, tgt)

RuntimeError: the feature number of src and tgt must be equal to d_model

In [70]:
src = torch.rand((10, 32, 32))
decoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=32, nhead=8, dropout=0.1),num_layers=2)
decoder(src).shape

torch.Size([10, 32, 32])

In [73]:
encoder = nn.TransformerEncoderLayer(d_model=32, nhead=8, dropout=0.1)
encoder(src).shape

torch.Size([10, 32, 32])

In [103]:
m = nn.AdaptiveAvgPool2d((4,1))
input = torch.randn(2,3,4,8)
output = m(input).squeeze(-1)
print(output.shape)
output = nn.Linear(4,1)(output)
print(output.shape)

torch.Size([2, 3, 4])
torch.Size([2, 3, 1])


In [106]:
input = torch.randn(2,3,4,8)
output = nn.Linear(8,1)(input)
print(output.shape)
output = nn.Linear(4,1)(output.squeeze(-1))
print(output.shape)

torch.Size([2, 3, 4, 1])
torch.Size([2, 3, 1])


In [110]:
from sklearn.preprocessing import StandardScaler
data = [[1,1], [2,3], [3,2], [1,1]]
scaler = StandardScaler()
scaled = scaler.fit_transform(data)
print(scaled)

# for inverse transformation
inversed = scaler.inverse_transform(scaled)
print(inversed)

[[-0.90453403 -0.90453403]
 [ 0.30151134  1.50755672]
 [ 1.50755672  0.30151134]
 [-0.90453403 -0.90453403]]
[[1. 1.]
 [2. 3.]
 [3. 2.]
 [1. 1.]]


In [116]:
data1 = utils.data.TensorDataset(src, src, src)
data1

In [120]:
torch.logical_not(torch.tensor([0, 1, -10])==0)

tensor([False,  True,  True])